In [ ]:
# Clone the repo
!git clone https://github.com/thunlp/StyleAttack.git

In [ ]:
# Move to correct directory
!pwd
%cd /content/StyleAttack/experiments
!pwd

In [ ]:
# Install requirements
!pwd
!pip3 install torch
!pip3 install transformers

In [ ]:
# Two tricks, MT and Aug
tricks = ["run_poison_bert_mt.py", "run_poison_bert_aug.py"]
# Three datasets
datasets = ["sst-2", "hate-speech", "ag"]
# Three BERT models
models = ["bert-base-uncased", "distilbert-base-uncased", "roberta-base"]
# Five poison styles present in their datasets
styles = ["bible", "lyrics", "poetry", "shakespeare", "tweets"]  

# Format string script to prepare the data
# 0 = dataset, 1 = style
sh = """
python prepare_probingdata.py --data {0} --transfer_type {1} --transfer_data_base_path ../data/transfer/{1}/{0} --orig_data_path ../data/clean/{0}
"""

# Format string script to actually run the model
# 0 = trick, 1 = dataset, 2 = model, 3 = style
sh2 = """
CUDA_VISIBLE_DEVICES=0 python {0} --data {1} --transferdata_path ../data/transfer/{3}/{1} --origdata_path ../data/clean/{1} --transfer_type {3} --bert_type {2} --output_num 2 --poison_method dirty --poison_rate 1 --blend False --transfer False 
"""

for dataset in datasets:
  for style in styles:
    # Prepare the data
    with open('script.sh', 'w') as file:
          file.write(sh.format(dataset, style))
    !bash script.sh

    for trick in tricks:
      for model in models:
        with open('script2.sh', 'w') as file:
          file.write(sh2.format(trick, dataset, model, style))

        # Run the model and label output
        print(f"Currently running: {trick} / {dataset} / {model} / {style}")
        !bash script2.sh
        print()
        print()